In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.tensorboard import SummaryWriter
import tqdm
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [9]:
train_data_path="C:/Users/alejg/Documents/proyectos_personales/image_classification/cat-breeds-dataset2GB/train/"
val_data_path="C:/Users/alejg/Documents/proyectos_personales/image_classification/cat-breeds-dataset2GB/test/"
#train_data_path="C:/Users/alejg/Documents/proyectos_personales/image_classification/dogs1GB/dogImages/dogImages/train/"
#val_data_path="C:/Users/alejg/Documents/proyectos_personales/image_classification/dogs1GB/dogImages/dogImages/valid/"
num_epochs = 30
epoch_ini=0
print(len(os.listdir(train_data_path)))
tranforms = transforms.Compose([
    transforms.Resize((100,100)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

66


In [10]:
import os
len(os.listdir(train_data_path))

66

In [11]:
#class Net2(nn.Module):
#
#    def __init__(self):
#        super(Net2, self).__init__()
#        self.conv1 = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=4)#->224->221->110 64->61->30
#        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3)#110->108->54   30->28->14
#        self.conv3 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=2)#54->53->26    14->13->6
#        self.conv4 = nn.Conv2d(in_channels=128,out_channels=256,kernel_size=1)#26->26->13   6->6->3
#        self.pool = nn.MaxPool2d(2,2)
#        self.drop1 = nn.Dropout(p=0.1)
#        self.drop2 = nn.Dropout(p=0.2)
#        self.drop3 = nn.Dropout(p=0.3)
#        self.drop4 = nn.Dropout(p=0.4)
#        self.drop5 = nn.Dropout(p=0.5)
#        self.drop6 = nn.Dropout(p=0.6)
#        self.dense1=nn.Linear(6400, 1000)#13*13*256
#        self.dense2=nn.Linear(1000, 1000)
#        self.dense3=nn.Linear(1000, len(os.listdir(train_data_path)))
#        #---
#
#        
#    def forward(self, x):
#        x = self.drop1(self.pool(F.relu(self.conv1(x))))
#        x = self.drop2(self.pool(F.relu(self.conv2(x))))
#        x = self.drop3(self.pool(F.relu(self.conv3(x))))
#        x = self.drop4(self.pool(F.relu(self.conv4(x))))
#
#        x = x.view(x.size(0), -1)
#
#        x = self.drop5(F.relu(self.dense1(x)))
#        x = self.drop6(F.relu(self.dense2(x)))
#        x = self.dense3(x)
#
#        return x

In [12]:
#model = Net2()

In [13]:
model = models.vgg16(pretrained=True)


In [14]:
for param in model.parameters():
    param.requires_grad = False

In [15]:
#https://towardsdatascience.com/batch-normalization-and-dropout-in-neural-networks-explained-with-pytorch-47d7a8459bcd
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          
                          ('fc1', nn.Linear(25088, 500)),
                          ('relu', nn.ReLU()),
                          ('drop1', nn.Dropout(p=0.65)),
                          ('fc2', nn.Linear(500, len(os.listdir(train_data_path)))),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [16]:
len(os.listdir(train_data_path))

66

In [17]:
os.makedirs("saved_models", exist_ok=True)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device



device(type='cuda')

In [18]:
print(f"epoch_ini= {epoch_ini}")
if epoch_ini != 0:
    # Load pretrained models
    
    model.load_state_dict(torch.load("saved_models/generator_%d.pth" % epoch_ini))
    epoch_ini=epoch_ini+1
    print("loaded")
#else:
#    model = Net2()

epoch_ini= 0


In [12]:
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [13]:
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)


In [14]:
def mi_validation(model, loader, criterion):
    test_loss = 0
    accuracy = 0
    for ii, (inputs, labels) in enumerate(tqdm.tqdm(loader)):
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        output = model.forward(inputs)
        test_loss += criterion(output, labels).item()

        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
        
    return test_loss, accuracy



In [4]:
train_data=torchvision.datasets.ImageFolder(root=train_data_path,
                                            transform=tranforms)

In [5]:
val_data = torchvision.datasets.ImageFolder(root=val_data_path,
                                            transform=tranforms)

In [6]:
batch_size=512
train_data_loader = DataLoader(train_data, batch_size=batch_size,
                               shuffle=True)
val_data_loader  = DataLoader(val_data, batch_size=batch_size,
                              shuffle=True)


In [18]:
from torch.utils.tensorboard import SummaryWriter
#tensorboard --logdir=C:/Users/alejg/Documents/tfm/image_classifier_backend/train/tensroboar


In [7]:
len(train_data_loader)

14

In [8]:
len(val_data_loader)

2

In [22]:

writer = SummaryWriter('tensroboar')
model.to('cuda')
running_loss = 0
for epoch in tqdm.tqdm(range(epoch_ini, num_epochs), desc="Training epoch"):
        print(epoch)
        model.train()
        
        
        
        for i, (inputs, labels) in enumerate(tqdm.tqdm(train_data_loader, desc="Training")):
            print(f"dataloade mini batch: {i}")
            inputs, labels = inputs.to('cuda'), labels.to('cuda')
        
            optimizer.zero_grad()
        
            # Forward and backward passes
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
            running_loss += loss.item()
            
        writer.add_scalar('running_loss', running_loss, epoch)
        running_loss = 0
  
            
        model.eval()
        with torch.no_grad():
            train_loss, train_accuracy = mi_validation(model, train_data_loader, criterion)
            
        losstrain=train_loss/len(train_data_loader)
        acctrain=train_accuracy/len(train_data_loader)
        print(f"loss training {losstrain}")
        print(f"acc training {acctrain}")
        writer.add_scalar('acc_training', acctrain, epoch)
        writer.add_scalar('loss_training', losstrain, epoch)
        
        
        model.eval()
        with torch.no_grad():
            test_loss, accuracy = mi_validation(model, val_data_loader, criterion)
            
        losstest=test_loss/len(val_data_loader)
        acctest=accuracy/len(val_data_loader)
        print(f"loss validation {losstest}")
        print(f"acc validation{acctest}")
        writer.add_scalar('acc_validation', acctest, epoch)
        writer.add_scalar('loss_validation', losstest, epoch)
        
        torch.save(model.state_dict(), "saved_models/generator_%d.pth" % epoch)        

Training:   0%|                                                                                | 0/168 [00:00<?, ?it/s]

0
dataloade mini batch: 0



Training:   1%|▍                                                                       | 1/168 [00:01<03:51,  1.39s/it]

dataloade mini batch: 1



Training:   1%|▊                                                                       | 2/168 [00:02<03:53,  1.41s/it]

...

dataloade mini batch: 15



Training:  10%|██████▊                                                                | 16/168 [00:20<03:11,  1.26s/it]

dataloade mini batch: 16



Training epoch:  63%|██████████████████████████████████████▋                      | 19/30 [2:28:26<1:25:56, 468.75s/it]


KeyboardInterrupt: 

In [ ]:
epochs = 250
print_every = 50
steps = 0

# change to cuda
model.to('cuda')
running_loss = 0
for e in range(epochs):
    
    model.train()
    for ii, (inputs, labels) in enumerate(train_data_loader):
        steps += 1
        
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        
        optimizer.zero_grad()
        
        # Forward and backward passes
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            
            print("Epoch: {}/{}... ".format(e+1, epochs),
                  "Loss: {:.4f}".format(running_loss/print_every))
            model.eval()
            
            # Turn off gradients for validation, saves memory and computations
            with torch.no_grad():
                test_loss, accuracy = mi_validation(model, val_data_loader, criterion)
            
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Validation Loss: {:.3f}.. ".format(test_loss/len(val_data_loader)),
                  "Validation Accuracy: {:.3f}".format(accuracy/len(val_data_loader)))
            
            
            with torch.no_grad():
                test_loss, accuracy = mi_validation(model, val_data_loader, criterion)
            
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Validation Loss: {:.3f}.. ".format(test_loss/len(val_data_loader)),
                  "Validation Accuracy: {:.3f}".format(accuracy/len(val_data_loader)))
            
            
            
            running_loss = 0
            
            
            model.train()

In [ ]:
def predict_new_image(imgoriginal, model, threshold=0.7,device='cpu'):
    img=imgoriginal.copy()
    transform = transforms.Compose([
        transforms.Resize((100,100)), 
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    
    model.to('cpu')
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)
    model.eval()
    
    # forward pass
    out = model.forward(batch_t)
    train_data_path="C:/Users/alejg/Documents/proyectos_personales/image_classification/dogs1GB/dogImages/dogImages/train/"
    
    classes = os.listdir(train_data_path)
    _, index = torch.max(out, 1)
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
    jsonOutput={}
    if percentage[index[0]].item()>threshold:
        
        confianza=percentage[index[0]].item()
        if confianza > threshold:
            jsonOutput.update({
                                       'confidence':confianza,
                                       'label':classes[index[0]]}
                               )
        return jsonOutput
        
    



In [ ]:
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch
import base64
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
model = models.vgg16(pretrained=True)
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          
                          ('fc1', nn.Linear(25088, 500)),
                          ('relu', nn.ReLU()),
                          ('drop1', nn.Dropout(p=0.95)),
                          ('fc2', nn.Linear(500, 133)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier
model.load_state_dict(torch.load("saved_models/generator_%d.pth" % 124))

In [ ]:
path_test_images="D:/datos/train2017/train2017/000000002337.jpg"
threshold=0.7

with open(path_test_images, "rb") as imageFile:
    file = base64.b64encode(imageFile.read())

In [ ]:
from PIL import Image
import io
jpg_original = base64.b64decode(file)
image_data = jpg_original # byte values of the image
image = Image.open(io.BytesIO(image_data))
image.show()

In [ ]:
model.to('cpu')
model.eval()
predict_new_image(image, model, threshold=0.7,device='cpu')